### Load and Preprocess Images

In [4]:
# Set image dimensions
IMG_WIDTH, IMG_HEIGHT = 64, 64

# Load vehicle images and label them as 1
vehicle_paths = sorted(glob.glob('./VehicleImage/vehicles/*/*.png'))
vehicles = [cv2.resize(cv2.imread(img), (IMG_WIDTH, IMG_HEIGHT)) for img in vehicle_paths]
vehicle_labels = [1] * len(vehicles)

# Load non-vehicle images and label them as 0
nonvehicle_paths = sorted(glob.glob('./VehicleImage/non-vehicles/*/*.png'))
nonvehicles = [cv2.resize(cv2.imread(img), (IMG_WIDTH, IMG_HEIGHT)) for img in nonvehicle_paths]
nonvehicle_labels = [0] * len(nonvehicles)

# Combine data and labels
X = np.array(vehicles + nonvehicles)
y = np.array(vehicle_labels + nonvehicle_labels)

### split the data

In [5]:
from sklearn.model_selection import train_test_split

# Initial split: Training and temporary set (combined validation + test)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
    
# Second split: Validation and training
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)
# 0.1765 * 0.85 ~= 0.15, so we end up with 70% train, 15% val, 15% test

# Normalize the images
X_train, X_val, X_test = X_train / 255.0, X_val / 255.0, X_test / 255.0

# Convert labels to categorical format
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense

# Denoising CNN (Autoencoder)
def build_denoise_cnn(input_shape):
    input_img = Input(shape=input_shape)
    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    # Output
    decoded = Conv2D(input_shape[2], (3, 3), activation='sigmoid', padding='same')(x)
    denoise_model = Model(inputs=input_img, outputs=decoded)
    return denoise_model

# Object Detection CNN
def build_object_detection_cnn(input_shape):
    input_img = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu')(input_img)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)
    detection_model = Model(inputs=input_img, outputs=output)
    return detection_model

# Combine Denoising and Object Detection into a Hybrid Model
def build_hybrid_model(input_shape):
    # Build the denoising model
    denoise_model = build_denoise_cnn(input_shape)
    # Build the object detection model
    detection_model = build_object_detection_cnn(input_shape)
    # Input layer
    input_img = Input(shape=input_shape)
    # Pass the input through the denoising model
    denoised_img = denoise_model(input_img)
    # Pass the denoised image to the detection model
    output = detection_model(denoised_img)
    # Hybrid model
    hybrid_model = Model(inputs=input_img, outputs=output)
    return hybrid_model

# Define input shape
input_shape = (64, 64, 3)

# Build the hybrid model
hybrid_model = build_hybrid_model(input_shape)

# Compile the model
hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
hybrid_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ (None, 64, 64, 3)      │        11,011 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_4 (Functional)       │ (None, 2)              │     3,287,298 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,298,309 (12.58 MB)

 Trainable params: 3,298,309 (12.58 MB)

 Non-trainable params: 0 (0.00 B)

### Train the model

In [ ]:
# Ensure the model is defined by running the cell where the model is created and compiled
history = hybrid_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/50


2024-10-30 15:45:32.190289: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 51/161 ━━━━━━━━━━━━━━━━━━━━ 27:51 15s/step - accuracy: 0.5502 - loss: 0.8903

In [ ]:
     def build_denoise_cnn(input_shape):
         input_img = Input(shape=input_shape)

         # Encoder
         x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
         x = MaxPooling2D((2, 2), padding='same')(x)
         x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
         x = MaxPooling2D((2, 2), padding='same')(x)

         # Decoder
         x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
         x = UpSampling2D((2, 2))(x)
         x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
         x = UpSampling2D((2, 2))(x)

         # Output
         decoded = Conv2D(input_shape[2], (3, 3), activation='sigmoid', padding='same')(x)

         denoise_model = Model(inputs=input_img, outputs=decoded)
         return denoise_model
     
      # Enhancing the Object Detection CNN:

     def build_detection_cnn(input_shape):
         input_img = Input(shape=input_shape)
         x = Conv2D(64, (3, 3), activation='relu')(input_img)
         x = BatchNormalization()(x)
         x = MaxPooling2D((2, 2))(x)
         x = Conv2D(128, (3, 3), activation='relu')(x)
         x = BatchNormalization()(x)
         x = MaxPooling2D((2, 2))(x)
         x = Flatten()(x)
         x = Dense(256, activation='relu')(x)
         x = Dropout(0.5)(x)
         output = Dense(2, activation='softmax')(x)
         detection_model = Model(inputs=input_img, outputs=output)
         return detection_model

    # Using a Pre-trained Model for Detection

     from tensorflow.keras.applications import ResNet50

     def build_detection_cnn(input_shape):
         base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
         base_model.trainable = False  # Freeze base model

         x = base_model.output
         x = Flatten()(x)
         x = Dense(256, activation='relu')(x)
         x = Dropout(0.5)(x)
         output = Dense(2, activation='softmax')(x)

         detection_model = Model(inputs=base_model.input, outputs=output)
         return detection_model

# Building the Hybrid Model:

   def build_hybrid_model(input_shape):
       denoise_model = build_denoise_cnn(input_shape)
       detection_model = build_detection_cnn(input_shape)

       input_img = Input(shape=input_shape)
       denoised_img = denoise_model(input_img)
       output = detection_model(denoised_img)

       hybrid_model = Model(inputs=input_img, outputs=output)
       return hybrid_model

   # Instantiate and compile the hybrid model
   hybrid_model = build_hybrid_model((64, 64, 3))
   hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
